In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt
import numpy as np

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.fashion_mnist.load_data()
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

4422102/4422102 [==============================] - 1s 0us/step


((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [3]:
# 라벨 설정
label_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [4]:
# 채널 값을 부여 : 28, 28 -> 28, 28, 1
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)
train_images.shape, test_images.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [5]:
# AlexNet 모델 입력층에 맞게 채널 3장으로 맞춤
train_images = tf.repeat(train_images, 3, axis=3)
test_images = tf.repeat(test_images, 3, axis=3)
train_images.shape, test_images.shape

(TensorShape([60000, 28, 28, 3]), TensorShape([10000, 28, 28, 3]))

In [6]:
#모델 구축
model = models.Sequential()
## 사용할 layer module
# 227 x 227 x 3으로 resize
model.add(layers.experimental.preprocessing.Resizing(227, 227,
                                                     input_shape=train_images.shape[1:]))

model.add(layers.Conv2D(96, (11, 11), activation='relu', strides = 4))#1 imagesize에 맞게 큰 filter에 큰 stride 적용
model.add(layers.Lambda(tf.nn.local_response_normalization)) #relu 결과값 안정화
model.add(layers.MaxPooling2D((3, 3), strides = 2)) #overlapping Pooling

model.add(layers.Conv2D(256, (5, 5), activation='relu', padding = 'same'))#2
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Conv2D(384, (3, 3), activation='relu', padding = 'same'))#3

model.add(layers.Conv2D(384, (3, 3), activation='relu', padding = 'same'))#4

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding = 'same'))#5
model.add(layers.MaxPooling2D((3, 3)))

model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu')) #6 완전연결계층
model.add(layers.Dropout(0.5))

model.add(layers.Dense(4096, activation='relu')) #7 완전연결계층
model.add(layers.Dropout(0.5))

model.add(layers.Dense(1000, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 227, 227, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 55, 55, 96)        34944     
                                                                 
 lambda (Lambda)             (None, 55, 55, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 256)       614656    
                                                                 
 lambda_1 (Lambda)           (None, 27, 27, 256)       0         
                                                        

In [ ]:
model = models.Sequential()

# 227 x 227 x 3으로 resize
model.add(layers.experimental.preprocessing.Resizing(227, 227,
                                                     input_shape=train_images.shape[1:]))


# kernel 개수, 크기, stride,
# conv, pool(overlap), LRN, FC, dropout
# output shape : 1000 -> 10

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_5 (Resizing)       (None, 227, 227, 1)       0         
                                                                 
 conv2d_25 (Conv2D)          (None, 55, 55, 96)        11712     
                                                                 
 lambda_10 (Lambda)          (None, 55, 55, 96)        0         
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 27, 27, 96)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 27, 27, 256)       614656    
                                                                 
 lambda_11 (Lambda)          (None, 27, 27, 256)       0         
                                                      

In [8]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
375/375 [==============================] - 173s 418ms/step - loss: 0.9299 - accuracy: 0.6817 - val_loss: 0.4284 - val_accuracy: 0.8426
Epoch 2/10
375/375 [==============================] - 157s 419ms/step - loss: 0.3700 - accuracy: 0.8608 - val_loss: 0.3218 - val_accuracy: 0.8839
Epoch 3/10
375/375 [==============================] - 157s 419ms/step - loss: 0.3155 - accuracy: 0.8825 - val_loss: 0.3130 - val_accuracy: 0.8797
Epoch 4/10
375/375 [==============================] - 157s 419ms/step - loss: 0.2854 - accuracy: 0.8921 - val_loss: 0.2994 - val_accuracy: 0.8923
Epoch 5/10
375/375 [==============================] - 148s 396ms/step - loss: 0.2702 - accuracy: 0.8999 - val_loss: 0.2818 - val_accuracy: 0.8957
Epoch 6/10
375/375 [==============================] - 157s 419ms/step - loss: 0.2519 - accuracy: 0.9040 - val_loss: 0.2763 - val_accuracy: 0.9013
Epoch 7/10
375/375 [==============================] - 157s 419ms/step - loss: 0.2337 - accuracy: 0.9115 - val_loss: 0.2652 -

In [ ]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 10s 30ms/step - loss: 0.2822 - accuracy: 0.9016


[0.2822279930114746, 0.9016000032424927]

In [ ]:
# AlexNet 모델을 간소화해봅시다
## resize없이 (28, 28, 3)으로 받아봅시다
model = models.Sequential()
## ** activation 은 모두 relu로 유지
model.add(layers.Conv2D(32, (3, 3), strides=1, activation='relu', input_shape=(28, 28, 3)))
model.add(layers.MaxPooling2D((2, 2)))
## layer 1 : 32개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
## layer 2 : 64개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x / normalizing 유지
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
## layer 3 : 128개, (3, 3), strides=1 / pooling은 (2, 2) overlapping x

## layer 4, 5 : 삭제

## fully connected layer 1, 2 : node 원하는대로 주기
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))